In [1]:
import pandas as pd

In [2]:
# import google analytics pages data
ga_page_data = pd.read_csv('D:\Python for Marketing\Ex_Files_Python_Marketing\Ex_Files_Python_Marketing\Exercise Files\data\ga_data\ga_pages.csv',skiprows=6, nrows=376)

In [3]:
# importing pages data from google search console
gsc_page = pd.read_csv('D:\Python for Marketing\Ex_Files_Python_Marketing\Ex_Files_Python_Marketing\Exercise Files\data\gsc_data\gsc_pages.csv')

In [4]:
# import Facebook ad data
fd_data = pd.read_csv('D:\\Python for Marketing\\Ex_Files_Python_Marketing\\Ex_Files_Python_Marketing\\Exercise Files\\data\\other_data\\facebook_data_ads.csv',
                     parse_dates = ['Reporting starts','Reporting ends'])

## Fix google Analytics Page data 

In [5]:
# Look at Google Analytics data
ga_page_data.head()

,Page,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,/,813,665,00:02:05,641,14.35%,55.97%,£0.00
1,/www.miratrix.co.uk,515,388,00:02:17,373,59.25%,56.89%,£0.00
2,/get-in-touch/,336,163,00:01:06,23,10.53%,33.04%,£0.00
3,/app-marketing-agency/,140,120,00:02:04,73,18.31%,57.86%,£0.00
4,/app-store-optimization-services/,136,105,00:01:30,22,14.29%,47.79%,£0.00


## Recreate Google Analytics page URL 

In [8]:
# Split the url by /
split_dom = ga_page_data.Page.str.rpartition("/")

In [9]:
split_dom.head()

,0,1,2
0,,/,
1,,/,www.miratrix.co.uk
2,/get-in-touch,/,
3,/app-marketing-agency,/,
4,/app-store-optimization-services,/,


In [10]:
split_dom[2].replace("", "miratrix.co.uk", inplace = True)

In [11]:
# change URL format so that it matches other data sources
split_dom[2].replace('www.miratrix.co.uk','miratrix.co.uk',inplace = True)

In [12]:
split_dom.head()

,0,1,2
0,,/,miratrix.co.uk
1,,/,miratrix.co.uk
2,/get-in-touch,/,miratrix.co.uk
3,/app-marketing-agency,/,miratrix.co.uk
4,/app-store-optimization-services,/,miratrix.co.uk


In [13]:
ga_page_data.Page = 'https://' + split_dom[2] + split_dom[0] + "/"

In [14]:
ga_page_data.head()

,Page,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,https://miratrix.co.uk/,813,665,00:02:05,641,14.35%,55.97%,£0.00
1,https://miratrix.co.uk/,515,388,00:02:17,373,59.25%,56.89%,£0.00
2,https://miratrix.co.uk/get-in-touch/,336,163,00:01:06,23,10.53%,33.04%,£0.00
3,https://miratrix.co.uk/app-marketing-agency/,140,120,00:02:04,73,18.31%,57.86%,£0.00
4,https://miratrix.co.uk/app-store-optimization-...,136,105,00:01:30,22,14.29%,47.79%,£0.00


## Grouping Data 

In [15]:
ga_page_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Page               376 non-null    object
 1   Pageviews          376 non-null    int64 
 2   Unique Pageviews   376 non-null    int64 
 3   Avg. Time on Page  376 non-null    object
 4   Entrances          376 non-null    int64 
 5   Bounce Rate        376 non-null    object
 6   % Exit             376 non-null    object
 7   Page Value         376 non-null    object
dtypes: int64(3), object(5)
memory usage: 23.6+ KB


## Wrangling data types for Groupby function 

In [16]:
# Replace 0 with time format 00:00:00
ga_page_data['Avg. Time on Page'] = ga_page_data['Avg. Time on Page'].replace(0, '00:00:00')

In [18]:
# convert column from string to time delta
ga_page_data['Avg. Time on Page'] = pd.to_timedelta(ga_page_data['Avg. Time on Page'])

In [19]:
#Replace % with nothing
ga_page_data['Bounce Rate']= ga_page_data['Bounce Rate'].str.replace('%',"")
#convert the column into a float
ga_page_data['Bounce Rate']= ga_page_data['Bounce Rate'].astype('float')

In [20]:
# Replace % with nothing
ga_page_data['% Exit']= ga_page_data['% Exit'].str.replace('%',"")
#convert the column into a float
ga_page_data['% Exit']= ga_page_data['% Exit'].astype('float')

In [21]:
# Replace % with nothing
ga_page_data['Page Value']= ga_page_data['Page Value'].str.replace('£',"")
#convert the column into a float
ga_page_data['Page Value']= ga_page_data['Page Value'].astype('float')

In [23]:
# Set the index for the dataframe to page
ga_page_data.set_index('Page', inplace = True)

In [24]:
ga_page_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, https://miratrix.co.uk/ to https://www.miratrix.co.uk#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   Pageviews          376 non-null    int64          
 1   Unique Pageviews   376 non-null    int64          
 2   Avg. Time on Page  376 non-null    timedelta64[ns]
 3   Entrances          376 non-null    int64          
 4   Bounce Rate        376 non-null    float64        
 5   % Exit             376 non-null    float64        
 6   Page Value         376 non-null    float64        
dtypes: float64(3), int64(3), timedelta64[ns](1)
memory usage: 23.5+ KB


## Applying Groupby to the data 

In [26]:
# Groupby Index and get the mean of the 'Avg. Time on Page'
avg_time = ga_page_data.groupby(ga_page_data.index)['Avg. Time on Page'].mean(numeric_only = False)

In [27]:
# Converting avg time to dataframe and store as ga_Time
ga_time = pd.DataFrame(avg_time)

In [31]:
# Groupby bounce, exit, pageview and Unique pageview using the appropriate math
# and store in a dataframe. Save as ga_bou_ex_type
ga_bou_ex_type = pd.DataFrame(ga_page_data.groupby(ga_page_data.index).agg({
'Bounce Rate':['mean'],'% Exit':['mean'],'Pageviews': sum, 'Unique Pageviews': sum}))

In [33]:
ga_time['Avg. Time on Page'].head()

Page
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/                                                  0 days 00:00:19
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/app-marketing-agency/                             0 days 00:00:04
https://+&cd=10&hl=en&ct=clnk&gl=in/search?q=cache:bI4Yk20ZHV8J:https://miratrix.co.uk/get-in-touch/                0 days 00:00:00
https://+&cd=5&hl=en&ct=clnk&gl=uk/search?q=cache:CAP7TiQpKL4J:https://miratrix.co.uk/tag/bridgnorth-shoplifters/   0 days 00:00:00
https://39/blog/page/                                                                                               0 days 00:00:00
Name: Avg. Time on Page, dtype: timedelta64[ns]

In [32]:
ga_time.head()

,Avg. Time on Page
Page,
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/,0 days 00:00:19
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/app-marketing-agency/,0 days 00:00:04
https://+&cd=10&hl=en&ct=clnk&gl=in/search?q=cache:bI4Yk20ZHV8J:https://miratrix.co.uk/get-in-touch/,0 days 00:00:00
https://+&cd=5&hl=en&ct=clnk&gl=uk/search?q=cache:CAP7TiQpKL4J:https://miratrix.co.uk/tag/bridgnorth-shoplifters/,0 days 00:00:00
https://39/blog/page/,0 days 00:00:00


In [34]:
ga_bou_ex_type.head()

,Bounce Rate,% Exit,Pageviews,Unique Pageviews
,mean,mean,sum,sum
Page,,,,
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/,0.0,50.0,2,2
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/app-marketing-agency/,0.0,50.0,2,2
https://+&cd=10&hl=en&ct=clnk&gl=in/search?q=cache:bI4Yk20ZHV8J:https://miratrix.co.uk/get-in-touch/,0.0,100.0,1,1
https://+&cd=5&hl=en&ct=clnk&gl=uk/search?q=cache:CAP7TiQpKL4J:https://miratrix.co.uk/tag/bridgnorth-shoplifters/,100.0,100.0,1,1
https://39/blog/page/,0.0,100.0,1,1


In [35]:
ga_bou_ex_type.info()

<class 'pandas.core.frame.DataFrame'>
Index: 316 entries, https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/ to https://xFRAx2jfTOmg+6EPY+09e3SpRN05tZm6PA=&b=5&f=frame&u=quh9O6fk3cp+g1Q0B84ejuJAQEZJll4HdKCQfOVJedw=&b=5/go.php?u=quh9O7m+gcpnkEcjR8wG07lIAEwDjhYPfuuLYf1GaJpICjY4ebQy5itrwJGI8mdy/xFTBhuneDOmg+6EPY+09e3SpRN05tZo5/w=&b=5&f=frame&u=quh9O7m+gcpnkEcjR8wG07lIAEwDjhYPfuuLYf1GaJpICjY4ebQy5itrwJGI8mdy/
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (Bounce Rate, mean)      316 non-null    float64
 1   (% Exit, mean)           316 non-null    float64
 2   (Pageviews, sum)         316 non-null    int64  
 3   (Unique Pageviews, sum)  316 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 12.3+ KB


## Rebuilding the google analytics dataframe 

In [41]:
# Merge ga_time and ga_bou_ex_type on page and with an inner join
new_ga = pd.merge(ga_time, # Left dataset
                 ga_bou_ex_type, # right dataframe
                 how = 'inner', # we are joining using the data in the column
                 left_on = 'Page', # the left dataset is being joined on the column page
                 right_on = 'Page', # the right dataset is being joined on the column page
                 ).sort_values(by='Avg. Time on Page',ascending = False) # sort the new dataframe

C:\Users\dell\AppData\Local\Temp\ipykernel_18944\3966042603.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  new_ga = pd.merge(ga_time, # Left dataset


In [42]:
new_ga.head()

,Avg. Time on Page,"(Bounce Rate, mean)","(% Exit, mean)","(Pageviews, sum)","(Unique Pageviews, sum)"
Page,,,,,
https://channels/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:21:45,0.0,0.0,1,1
https://miratrix.co.uk/top-3-competitor-analysis-for-aso/,0 days 00:18:09.500000,50.0,87.5,5,5
https://www.miratrix.co.uk?SuperSocializerAuth=LiveJournal/blog/page/30/,0 days 00:14:57,0.0,0.0,1,1
https://about/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:45,0.0,0.0,2,2
https://videos?view=0&sort=p&flow=grid/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:43,0.0,0.0,1,1


In [43]:
#rename (Bounce, rate, mean)
new_ga =  new_ga.rename(columns = {('Bounce Rate', 'mean'):'Bounce Rate',('% Exit','mean'):'Exit',
                                  ('(Pageviews', 'sum'):'Pageviews',
                                  ('Unique Pageviews', 'sum'):'Unique Pageviews'})

In [44]:
new_ga.head()

,Avg. Time on Page,Bounce Rate,Exit,"(Pageviews, sum)",Unique Pageviews
Page,,,,,
https://channels/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:21:45,0.0,0.0,1,1
https://miratrix.co.uk/top-3-competitor-analysis-for-aso/,0 days 00:18:09.500000,50.0,87.5,5,5
https://www.miratrix.co.uk?SuperSocializerAuth=LiveJournal/blog/page/30/,0 days 00:14:57,0.0,0.0,1,1
https://about/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:45,0.0,0.0,2,2
https://videos?view=0&sort=p&flow=grid/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:43,0.0,0.0,1,1


## Dropping Columns 

In [64]:
# check the Google Analytics Page data
ga_page_traffic_data = pd.read_csv('D:\Python for Marketing\Ex_Files_Python_Marketing\Ex_Files_Python_Marketing\Exercise Files\data\ga_data\ga_pages.csv',skiprows=386, nrows=367,
                                  parse_dates = ['Day Index'])

In [65]:
ga_page_traffic_data

,Day Index,Pageviews,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,2018-08-31,14,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-01,6,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-09-02,17,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-09-03,14,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-09-04,11,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
362,2019-08-28,8,NaN,NaN,NaN,NaN,NaN,NaN
363,2019-08-29,11,NaN,NaN,NaN,NaN,NaN,NaN
364,2019-08-30,11,NaN,NaN,NaN,NaN,NaN,NaN
365,2019-08-31,6,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
ga_page_traffic_data = ga_page_traffic_data[ga_page_traffic_data.columns[~ga_page_traffic_data.columns.isin(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7'])]]

In [67]:
ga_page_traffic_data.head()

,Day Index,Pageviews
0,2018-08-31,14
1,2018-09-01,6
2,2018-09-02,17
3,2018-09-03,14
4,2018-09-04,11


In [50]:
fd_data.head()

,Unnamed: 0,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,0,2014-02-21,2018-01-29,Campaign1,0,11451.0,NaN,1.275683,NaN,NaN,NaN,NaN,NaN,1807.0,42.0,133.0
1,1,2015-12-04,2015-12-04,Campaign2,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2,2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,NaN,3.530000,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,NaN,16.500000,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
4,4,2015-10-20,2015-10-20,Campaign4,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# drop all columns that have NaN
fd_data.drop('Unnamed: 0', axis =1 , inplace = True)

In [70]:
fd_data.head()

,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,2014-02-21,2018-01-29,Campaign1,0,11451.0,NaN,1.275683,NaN,NaN,NaN,NaN,NaN,1807.0,42.0,133.0
1,2015-12-04,2015-12-04,Campaign2,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,NaN,3.530000,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,NaN,16.500000,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
4,2015-10-20,2015-10-20,Campaign4,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Replaceing missing  (NaN) Facebook Ad data

In [71]:
fd_data.fillna(0,inplace = True)

In [72]:
#Check that fillna worked
fd_data.head()

,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,2014-02-21,2018-01-29,Campaign1,0,11451.0,0.0,1.275683,0.0,0.0,0.0,0.0,0.0,1807.0,42.0,133.0
1,2015-12-04,2015-12-04,Campaign2,inactive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,0.0,3.530000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,0.0,16.500000,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
4,2015-10-20,2015-10-20,Campaign4,inactive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Joining Google Analytics and Google Search Console Data 

In [75]:
# you pandas merge to join google console and google analytics data
ga_gsc_data = pd.merge(gsc_page,# left data
                      new_ga,# right data
                      how = 'inner',
                      left_on = 'Page',
                      right_index = True)

In [76]:
ga_gsc_data.head()

,Page,Clicks,Impressions,CTR,Position,Avg. Time on Page,Bounce Rate,Exit,"(Pageviews, sum)",Unique Pageviews
0,https://miratrix.co.uk/,371,100673,0.37%,42.59,0 days 00:01:43.666666666,47.296667,54.286667,1396,1095
1,https://miratrix.co.uk/app-marketing-agency/,80,30824,0.26%,26.84,0 days 00:01:45.333333333,64.436667,60.913333,210,181
2,https://miratrix.co.uk/mobile-app-competitor-a...,16,1458,1.1%,30.64,0 days 00:04:06,53.335000,89.475000,30,26
3,https://miratrix.co.uk/blippar-the-message-isn...,11,517,2.13%,36.05,0 days 00:00:22.333333333,58.170000,78.333333,27,21
4,https://miratrix.co.uk/author/nduddy/,10,140,7.14%,5.61,0 days 00:02:52,22.223333,38.890000,15,13


## Saving your data to a CSV 

In [77]:
# save the newly created page dataset
ga_gsc_data.to_csv('D:\Python for Marketing\Ex_Files_Python_Marketing\Ex_Files_Python_Marketing\Exercise Files\data\gsc_data\ga_gsc_data.csv')

In [79]:
fd_data.to_csv('D:\\Python for Marketing\\Ex_Files_Python_Marketing\\Ex_Files_Python_Marketing\\Exercise Files\\data\\other_data\\fixed_fb_data.csv')